In [1]:
import requests
import geopandas as gpd

# Dictionary of dataset names and their respective URLs
datasets = {
    "mci": "https://services.arcgis.com/S9th0jAJ7bqgIRjw/arcgis/rest/services/Major_Crime_Indicators_Open_Data/FeatureServer/0/query",
    "hom": "https://services.arcgis.com/S9th0jAJ7bqgIRjw/arcgis/rest/services/Homicides_Open_Data_ASR_RC_TBL_002/FeatureServer/0/query",
    "gun": "https://services.arcgis.com/S9th0jAJ7bqgIRjw/arcgis/rest/services/Shooting_and_Firearm_Discharges_Open_Data/FeatureServer/0/query",
    "bike": "https://services.arcgis.com/S9th0jAJ7bqgIRjw/arcgis/rest/services/Bicycle_Thefts_Open_Data/FeatureServer/0/query",
    "car":"https://services.arcgis.com/S9th0jAJ7bqgIRjw/arcgis/rest/services/Theft_From_Motor_Vehicle_Open_Data/FeatureServer/0/query",
    "nbr":"https://services.arcgis.com/S9th0jAJ7bqgIRjw/arcgis/rest/services/Neighbourhood_Crime_Rates_Open_Data/FeatureServer/0/query"
    # Add more datasets as needed
}

# Initialize a dictionary to store the GeoDataFrames
gdfs = {}

for dataset_name, url in datasets.items():
    all_features = []
    offset = 0
    while True:
        # Define request parameters
        params = {
            "where": "1=1",
            "outFields": "*",
            "outSR": "4326",
            "f": "geojson",
            "resultRecordCount": 2000,
            "resultOffset": offset
        }
        
        # Make the request
        response = requests.get(url, params=params)
        if response.status_code == 200:
            data = response.json()
            features = data.get("features", [])
            if not features:
                break  # Exit loop if no more features to fetch
            all_features.extend(features)
            offset += len(features)
        else:
            print(f"Failed to fetch data for {dataset_name}. Status code: {response.status_code}")
            break

    # Convert all_features to a GeoDataFrame
    gdf = gpd.GeoDataFrame.from_features(all_features)
    gdfs[f'gdf_{dataset_name}'] = gdf

# At this point, gdfs contains a GeoDataFrame for each dataset, accessible via its name, e.g., gdfs['gdf_dataset1']


In [4]:


print(gdfs['gdf_mci'].info())

<class 'geopandas.geodataframe.GeoDataFrame'>
RangeIndex: 372899 entries, 0 to 372898
Data columns (total 30 columns):
 #   Column             Non-Null Count   Dtype   
---  ------             --------------   -----   
 0   geometry           372899 non-null  geometry
 1   OBJECTID           372899 non-null  int64   
 2   EVENT_UNIQUE_ID    372899 non-null  object  
 3   REPORT_DATE        372899 non-null  int64   
 4   OCC_DATE           372899 non-null  int64   
 5   REPORT_YEAR        372899 non-null  int64   
 6   REPORT_MONTH       372899 non-null  object  
 7   REPORT_DAY         372899 non-null  int64   
 8   REPORT_DOY         372899 non-null  int64   
 9   REPORT_DOW         372899 non-null  object  
 10  REPORT_HOUR        372899 non-null  int64   
 11  OCC_YEAR           372788 non-null  float64 
 12  OCC_MONTH          372788 non-null  object  
 13  OCC_DAY            372788 non-null  float64 
 14  OCC_DOY            372788 non-null  float64 
 15  OCC_DOW            372788 